In [1]:
# pip install tensorflow

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings("ignore")

from PIL import Image

import tensorflow as tf
from tensorflow.keras.utils import load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
import random

In [5]:
print(tf.__version__)

2.12.0


In [6]:
benign_path = "Dataset_BUSI_with_GT/benign"
normal_path = "Dataset_BUSI_with_GT/normal"
malignant_path = "Dataset_BUSI_with_GT/malignant"
benign_dataset = []
normal_dataset = []
malignant_dataset = []

classes_path = [benign_path, normal_path, malignant_path]
classes_dataset = [benign_dataset, normal_dataset, malignant_dataset]

for index, _class in enumerate(classes_path):
    for filename in os.listdir(_class):
        image_path = os.path.join(_class, filename)
        
        if os.path.isfile(image_path) and filename.lower().endswith(').png'):
            mask_path = filename.replace('.png', '_mask.png')
            filename = os.path.join(_class, filename) 
            mask_path = os.path.join(_class, mask_path) 
            classes_dataset[index].append((filename ,mask_path))

In [7]:
color_yellow = '\033[33m' 
color_black = '\033[30m'

length = 0
for _class in classes_dataset:
    print('The length of the {} class is {}'.format(_class[0][0].split('/')[-2], len(_class)))
    length += len(_class)
print(color_yellow + 40*'-')
print(color_black + 'The total length of the dataset is ' + str(length))

The length of the benign class is 437
The length of the normal class is 133
The length of the malignant class is 210
----------------------------------------
The total length of the dataset is 780


In [8]:
classes_dataset = [path for sublist in classes_dataset for path in sublist]

In [9]:
classes_dataset[1]

('Dataset_BUSI_with_GT/benign/benign (147).png',
 'Dataset_BUSI_with_GT/benign/benign (147)_mask.png')

In [19]:
# image = cv2.imread(classes_dataset[1][0])
# mask = cv2.imread(classes_dataset[1][1]) 

In [32]:
# added_image = cv2.addWeighted(image,0.6,mask,0.1,0)

# cv2.imwrite('combined.png', added_image)

True

In [30]:
# image = cv2.imread('combined.png')

In [11]:

class UNet:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()

    def conv_block(self, inputs, filters, kernel_size=3, activation='relu', padding='same'):
        conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
        conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(conv)
        return conv

    def build_model(self):
        inputs = Input(self.input_shape)

        # Encoder
        conv1 = self.conv_block(inputs, 64)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = self.conv_block(pool1, 128)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = self.conv_block(pool2, 256)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = self.conv_block(pool3, 512)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = self.conv_block(pool4, 1024)

        # Decoder
        up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
        concat6 = concatenate([conv4, up6], axis=3)
        conv6 = self.conv_block(concat6, 512)

        up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
        concat7 = concatenate([conv3, up7], axis=3)
        conv7 = self.conv_block(concat7, 256)

        up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
        concat8 = concatenate([conv2, up8], axis=3)
        conv8 = self.conv_block(concat8, 128)

        up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
        concat9 = concatenate([conv1, up9], axis=3)
        conv9 = self.conv_block(concat9, 64)

        # Output
        outputs = Conv2D(self.num_classes, (1, 1), activation='softmax')(conv9)

        model = Model(inputs=inputs, outputs=outputs)
        return model


In [13]:
input_shape = (256, 256, 3)
num_classes = 3

unet = UNet(input_shape, num_classes)
model = unet.model
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_76 (Conv2D)             (None, 256, 256, 64  1792        ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_77 (Conv2D)             (None, 256, 256, 64  36928       ['conv2d_76[0][0]']              
                                )                                                           

                                )                                                                 
                                                                                                  
 conv2d_93 (Conv2D)             (None, 256, 256, 64  36928       ['conv2d_92[0][0]']              
                                )                                                                 
                                                                                                  
 conv2d_94 (Conv2D)             (None, 256, 256, 3)  195         ['conv2d_93[0][0]']              
                                                                                                  
Total params: 31,031,875
Trainable params: 31,031,875
Non-trainable params: 0
__________________________________________________________________________________________________
